In [1]:
import os
import json
import pandas as pd
from datetime import datetime
from collections import defaultdict

In [2]:
path_dados = 'data/13-10-2020/'

In [3]:
def abrir_json(path):
    
    with open(path, 'r', encoding = 'utf-8') as f:
        content = f.read()
    return json.loads(content)


In [4]:
requisicoes = abrir_json(path_dados+'consulta.json')

In [5]:
procs = abrir_json(path_dados+'registro.json')

In [6]:
def checar_retorno_ok(resp):
    
    return resp.get('codRetorno', 9999) == 0

def checar_se_fechou(req):
    
    situ_arrecad = req.get('ObterSituacaoFechamentoArrecadacao', {}).get('response', {})
    if checar_retorno_ok(situ_arrecad):
        cod_situ = situ_arrecad['codSituacao']
        return cod_situ == 2
    
    return False


def convert_timestamp(timestamp):
    
    arrumada_miliseconds_js = timestamp/1000
    
    return datetime.fromtimestamp(arrumada_miliseconds_js)


def checar_se_no_periodo(timestamp, mes, ano):
    
    dt_object = convert_timestamp(timestamp)
    
    return dt_object.year == ano and dt_object.month == mes

def checar_conciliou(req):
    
    demonstrativo = req.get('ObterDemonstrativoArrecadaoDiaria', {})
    demonstrativo = demonstrativo.get('response', {}).get('demonstrativo', {})
    pagamentos = req.get('ObterPagamentosPorDataArrecadacao', {}).get('response', {})
    if checar_retorno_ok(demonstrativo) and checar_retorno_ok(pagamentos):
        
        total_demonstrativo = demonstrativo.get('demonstrativo', {}).get('vlTotalPgtos', None)
        if total_demonstrativo: total_demonstrativo = float(total_demonstrativo)
        print(f'Total demonstrativo : {total_demonstrativo}')
        total_pagamentos = 0
        for pag in pagamentos.get('pagamentos', {}).get('pagamento',[]):
            total_pagamentos+= float(pag.get('vlPago', 0))
        print(f'Total pagamentos : {total_pagamentos}')
        return total_demonstrativo == total_pagamentos
    else:
        print(demonstrativo)
        
    return False

def pegar_pagamentos(req, mes, ano):
    
    pagamentos = []
    pags = req.get('ObterPagamentosPorDataArrecadacao', {}).get('response', {})
    if checar_retorno_ok(pags) and checar_conciliou(req):
        pags = pags['pagamentos']['pagamento']
        for pag in pags:
            if checar_se_no_periodo(pag['dtArrecadacao'], mes, ano):
                pagamentos.append(pag)
    return pagamentos
    

def pegar_boleto_proc(proc):
    
    bol =  proc['request']
    bol['numero_proc'] = proc['process']['nP']

    return bol


In [7]:
boletos = [pegar_boleto_proc(proc) for proc in procs]

In [8]:
pagamentos = [pag for req in requisicoes for pag in pegar_pagamentos(req, 9, 2020) ]

Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 192293.79
Total pagamentos : 192293.79
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 5536.79
Total pagamentos : 8589.99
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 90897.24
Total pagamentos : 90897.24
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 2472.53
Total pagamentos : 2472.53
Total demonstrativo : 28613.2
Total pagamentos : 28613.2
Total demonstrativo : 2771.48
Total pagamentos : 2771.48
Total demonstrativo : 21.5
Total pagamentos : 21.5
Total demonstrativo : 966.14
Total pagamentos : 966.14
Total demonstrativo : 8589.99
Total pagamentos : 8589.99
Total demonstrativo : 3097.89
Total pagamentos : 3097.89


In [9]:
req = requisicoes[0]

In [10]:
def adicionar_pags_aos_boletos(boletos, pagamentos):
    
    for boleto in boletos:
        boleto['pagamentos'] = []
        for pag in pagamentos:
            if int(boleto['numIndentificadorDocumento']) == int(pag['numIndentificadorDocumento']):
                boleto['pagamentos'].append(pag)
                
                
def adicionar_total_cobrado_boleto(boletos):
    
    for bol in boletos:
        bol['total_cobrado'] = 0

        for taxa in bol['reclassificacaoReceita']:
            bol['total_cobrado'] += taxa['vlrPrincipal']
            bol['total_cobrado'] += taxa['vlrMulta']
            bol['total_cobrado'] += taxa['vlrCorrecao']
            bol['total_cobrado'] -= taxa['vlrDesconto']
                
def adicionar_total_pago(boletos):
    
    for bol in boletos:
        bol['total_pago'] = 0
        for pag in bol['pagamentos']:
            bol['total_pago']+=pag['vlPago']

In [11]:
adicionar_pags_aos_boletos(boletos, pagamentos)

In [12]:
adicionar_total_cobrado_boleto(boletos)

In [13]:
adicionar_total_pago(boletos)

In [14]:
def gerar_status_pagamentos(boletos):
    
    for bol in boletos:
        if len(bol['pagamentos']) == 0:
            bol['status'] = 'Aguardando pagamento'
        else:
            if bol['total_pago'] == bol['total_cobrado']:
                bol['status'] = 'Quitado'
            elif bol['total_pago'] > bol['total_cobrado']:
                bol['status'] = 'A maior'
            elif bol['total_pago'] < bol['total_cobrado']:
                bol['status'] = 'A menor'
            
def checar_duplicatas(boletos):
    
    for bol in boletos:
        if len(bol['pagamentos']) > 1:
            bol['duplicadas'] = len(bol['pagamentos'])
        else:
            bol['duplicadas'] = None

In [15]:
gerar_status_pagamentos(boletos)
checar_duplicatas(boletos)

In [16]:
for bol in boletos:
    print(bol['status'])

Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Quitado
Aguardando pagamento
Aguardando pagamento
Aguardando pagamento
Aguardando pagamento
Aguardando pagamento


In [17]:
for bol in boletos:
    if bol['status'] == 'A maior':
        print(bol)
        print('____')

In [18]:
for bol in boletos:
    if bol['status'] == 'A Maior':
        try:
            assert bol['duplicadas']
        except:
            print(bol)

In [19]:
for bol in boletos:
    try:
        if bol['status'] == 'A Maior' and bol['duplicadas']:
            assert bol['total_pago'] == bol['total_cobrado']*bol['duplicadas']
    except:
        print(bol)

In [20]:
def valor_total_taxa(taxa):
    
    total=0
    total+= taxa['vlrPrincipal']
    total+= taxa['vlrMulta']
    total+= taxa['vlrCorrecao']
    total+=taxa['vlrMora']
    total-= taxa['vlrDesconto']
    
    return total
    

def gerar_rel_contabil(boletos):
    
    rel_contabil = defaultdict(int)
    
    for bol in boletos:
        if bol['status']=='Quitado':
            for taxa in bol['reclassificacaoReceita']:
                rel_contabil[taxa['tpoReclassificacao']] += valor_total_taxa(taxa)
        if bol['status']=='A Maior' and bol['duplicadas']:
            for taxa in bol['reclassificacaoReceita']:
                total_taxa = valor_total_taxa(taxa)*bol['duplicadas']
                rel_contabil[taxa['tpoReclassificacao']] += total_taxa
                
    return rel_contabil

In [21]:
rel_contabil = gerar_rel_contabil(boletos)

In [22]:
rel_contabil

defaultdict(int, {74: 448.5, 75: 325340.23})